## OpenGL SkyBox

In [ ]:
from math import modf
from math import *
from PIL import Image, ImageDraw
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *



def loadImage(imageName):
    im = Image.open(imageName)
    try:
        ix, iy, image = im.size[0], im.size[1], im.tobytes("raw", "RGB", 0, -1)
    except SystemError:
        ix, iy, image = im.size[0], im.size[1], im.tobytes("raw", "RGB", 0, -1)
    glEnable(GL_TEXTURE_2D)
    ID = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, ID)
    
    glTexEnvi(GL_TEXTURE_ENV, GL_TEXTURE_ENV_MODE, GL_MODULATE)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)

    glTexImage2D(GL_TEXTURE_2D, 0, 3, ix, iy, 0, GL_RGB, GL_UNSIGNED_BYTE, image)
    return ID

image = Image.open("map.bmp")

draw = ImageDraw.Draw(image) 

pix = image.load() 
global mapWidth, mapHeight

mapWidth = image.size[0]
mapHeight = image.size[1]

H = [[pix[y, x][0] for x in range(mapWidth)] for y in range(mapHeight)]

def norm(vec):
    l = sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    if vec != (0, 0, 0):
        return (vec[0] / l, vec[1] / l, vec[2] / l)
    else:
        return (0.0, 0.0, 0.0)
    
def VxV(v1, v2):
    return  (v1[1] * v2[2] - v1[2] * v2[1], v1[2] * v2[0] - v1[0] * v2[2], v1[0] * v2[1] - v1[1] * v2[0])

global skSize
skSize = 512.0

def init():
    global scrX, scrY
    scrX = glutGet(GLUT_SCREEN_WIDTH) // 8 * 3
    scrY = glutGet(GLUT_SCREEN_HEIGHT) // 8 * 3
    
    glEnable(GL_DEPTH_TEST)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(90, scrX/scrY , 0.001, 500)
    
    global posX, posY, posZ, dirX, dirY, dirZ, upX, upY, upZ, filled
    posX = 47
    posY = 47
    posZ = 50
    
    dirX = 47
    dirY = - 47
    dirZ = 0
    
    upX = 0
    upY = 0
    upZ = 1
    
    filled = 0
    
    global bk, dn, fd, lf, rt, up
    
    bk = loadImage("bk.tga")
    dn = loadImage("dn.tga")
    fd = loadImage("fd.tga")
    lf = loadImage("lf.tga")
    rt = loadImage("rt.tga")
    up = loadImage("up.tga")
    
def keyboardkeys(key, x, y):
    
    global posX, posY, posZ, dirX, dirY, dirZ, upX, upY, upZ
    
    if key == b'\x1b':
        sys.exit(0)
    if key == b'w':
        d = Norm((dirX, dirY, dirZ))
        posX += d[0]
        posY += d[1]
        posZ += d[2]
    if key == b'a':
        v = Norm(VxV((dirX, dirY, dirZ), (upX, upY, upZ)))
        posX -= v[0]
        posY -= v[1]
        posZ -= v[2]
    if key == b's':
        d = Norm((dirX, dirY, dirZ))
        posX -= d[0]
        posY -= d[1]
        posZ -= d[2]
    if key == b'd':
        v = Norm(VxV((dirX, dirY, dirZ), (upX, upY, upZ)))
        posX += v[0]
        posY += v[1]
        posZ += v[2]
    glutPostRedisplay()

def mouseButton(button, state, x, y):
    global filled
    if state == GLUT_DOWN:
        if button == GLUT_LEFT_BUTTON:
            filled = 1 - filled
        if button == GLUT_RIGHT_BUTTON:
            print('right')
        if button == 3:
            print('up')
        if button == 4:
            print('down')
            
    glutPostRedisplay()

def mouseMove(x, y):
    global dirX, dirY, dirZ, upX, upY, upZ, scrX, scrY
    
    dX = x - scrX
    dY = scrY - y
      
    dirX += dX
    dirZ += dY
    
    scrX = x
    scrY = y
    glutPostRedisplay()

def buildSkybox():
    global skSize, posX, posY, posZ, bk, rt, ft, lf, dn, up
    dist = skSize // 2
    glColor3f(1, 1, 1)
    glBindTexture(GL_TEXTURE_2D, fd)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 0.0); glVertex3f(posX - dist, posY + dist, posZ - dist)
    glTexCoord2f(1.0, 0.0); glVertex3f(posX + dist, posY + dist, posZ - dist)
    glTexCoord2f(1.0, 1.0); glVertex3f(posX + dist, posY + dist, posZ + dist)
    glTexCoord2f(0.0, 1.0); glVertex3f(posX - dist, posY + dist, posZ + dist)
    glEnd()
    
    glBindTexture(GL_TEXTURE_2D, lf)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 0.0); glVertex3f(posX + dist, posY + dist, posZ - dist)
    glTexCoord2f(1.0, 0.0); glVertex3f(posX + dist, posY - dist, posZ - dist)
    glTexCoord2f(1.0, 1.0); glVertex3f(posX + dist, posY - dist, posZ + dist)
    glTexCoord2f(0.0, 1.0); glVertex3f(posX + dist, posY + dist, posZ + dist)
    glEnd()
    
    glBindTexture(GL_TEXTURE_2D, rt)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 0.0); glVertex3f(posX - dist, posY - dist, posZ - dist)
    glTexCoord2f(1.0, 0.0); glVertex3f(posX - dist, posY + dist, posZ - dist)
    glTexCoord2f(1.0, 1.0); glVertex3f(posX - dist, posY + dist, posZ + dist)
    glTexCoord2f(0.0, 1.0); glVertex3f(posX - dist, posY - dist, posZ + dist)
    glEnd()
    
    glBindTexture(GL_TEXTURE_2D, bk)
    glBegin(GL_QUADS)
    glTexCoord2f(1.0, 0.0); glVertex3f(posX - dist, posY - dist, posZ - dist)
    glTexCoord2f(0.0, 0.0); glVertex3f(posX + dist, posY - dist, posZ - dist)
    glTexCoord2f(0.0, 1.0); glVertex3f(posX + dist, posY - dist, posZ + dist)
    glTexCoord2f(1.0, 1.0); glVertex3f(posX - dist, posY - dist, posZ + dist)
    glEnd()
    
    glBindTexture(GL_TEXTURE_2D, up)
    glBegin(GL_QUADS)
    glTexCoord2f(1.0, 0.0); glVertex3f(posX - dist, posY + dist, posZ + dist)
    glTexCoord2f(0.0, 0.0); glVertex3f(posX - dist, posY - dist, posZ + dist)
    glTexCoord2f(0.0, 1.0); glVertex3f(posX + dist, posY - dist, posZ + dist)
    glTexCoord2f(1.0, 1.0); glVertex3f(posX + dist, posY + dist, posZ + dist)
    glEnd()
    
    glBindTexture(GL_TEXTURE_2D, dn)
    glBegin(GL_QUADS)
    glTexCoord2f(1.0, 0.0); glVertex3f(posX - dist, posY + dist, posZ - dist)
    glTexCoord2f(0.0, 0.0); glVertex3f(posX - dist, posY - dist, posZ - dist)
    glTexCoord2f(0.0, 1.0); glVertex3f(posX + dist, posY - dist, posZ - dist)
    glTexCoord2f(1.0, 1.0); glVertex3f(posX + dist, posY + dist, posZ - dist)
    glEnd()
    
    
def buildLandscape():
    for y in range(1, len(H), 2):
        for x in range(1, len(H[0]), 2):
            
            glBegin(GL_TRIANGLE_FAN)
            glColor3f(1, 1, 1)
            
            glVertex3d(x, y, H[y][x])
            
            
            glVertex3d((x + 1), y, H[y][x + 1])
            glVertex3d((x + 1), (y + 1), H[y + 1][x + 1])
            glVertex3d(x, (y + 1), H[y + 1][x])
            glVertex3d((x - 1), (y + 1), H[y + 1][x - 1])
            glVertex3d((x - 1), y, H[y][x - 1])
            glVertex3d((x - 1), (y - 1), H[y - 1][x - 1])
            glVertex3d(x, (y - 1), H[y - 1][x])
            glVertex3d((x + 1), (y - 1), H[y - 1][x + 1])
            glVertex3d((x + 1), y, H[y][x + 1])
            
            glEnd()
    
 
def renderScene(*args, **kwargs):
    global posX, posY, posZ, dirX, dirY, dirZ, upX, upY, upZ
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) 
    glMatrixMode(GL_MODELVIEW) 
    glLoadIdentity()
     
    gluLookAt(posX, posY, posZ,       
        posX + dirX, posY + dirY, posZ + dirZ,          
        upX, upY, upZ)           
    
    if filled == 1:
        glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
    else:
        glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)

    glColor3f(0, 0, 0)
    glPushMatrix()
    buildLandscape()
    buildSkybox()
    glPopMatrix()
    
    glutSwapBuffers()           

    glutPostRedisplay()         
    
glutInit(sys.argv)
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
glutInitWindowSize(glutGet(GLUT_SCREEN_WIDTH) // 8 * 6, glutGet(GLUT_SCREEN_HEIGHT) // 8 * 6)
glutInitWindowPosition(glutGet(GLUT_SCREEN_WIDTH) // 8, glutGet(GLUT_SCREEN_HEIGHT) // 8)
glutCreateWindow(b"Free camera")
init()


glutDisplayFunc(renderScene)
glutKeyboardFunc(keyboardkeys)
glutPassiveMotionFunc(mouseMove)
glutMouseFunc(mouseButton)
glutSetCursor(3)
glutMainLoop()
